In [20]:
#import packages
import io
import string
import requests
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

In [127]:
# get bytes stream of web pdf
#pdfurl = "https://arxiv.org/pdf/2001.09903.pdf"
pdfurl = "http://arxiv.org/pdf/1811.04422v1"
#pdfurl = "https://arxiv.org/pdf/2001.09956"
#pdfurl = "https://arxiv.org/pdf/2001.09412.pdf"
#pdfurl = "http://arxiv.org/pdf/1411.6753v1"
#pdfurl = "https://arxiv.org/pdf/2001.10393.pdf"
r = requests.get(pdfurl, stream=True)
f = io.BytesIO(r.content)

In [128]:
# set up pdfminer
rsrcmgr = PDFResourceManager()
retstr = io.StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pagenos=set()

In [129]:
# extract abstract
content = []
cnt = 1
for page in PDFPage.get_pages(fp=f, pagenos=pagenos, maxpages=0, caching=True, check_extractable=True):
    interpreter.process_page(page)
    print(cnt)
    cnt += 1

text = retstr.getvalue()
content.append(text)
# close apps
device.close()
retstr.close()

1
2
3
4
5
6
7
8


In [130]:
text = []
splits = content[0].split("\n\n")
for chunk in splits:
    # if ocr has picked up annoying numbers along side with many "\n"
    if chunk.count("\n") >= 3:
        dummy = chunk.split("\n")
        dummy_cnt = 0
        for d in dummy:
            if len(d)>1:
                dummy_cnt += 1
        if dummy_cnt > 2:
            text.append(chunk)
    else:
        text.append(chunk)

In [131]:
# identify abstract
cnt = 0
while True:
    if "abstract" in text[cnt].lower():
        if len(text[cnt].split()) < 2:
            cnt += 1
            break
        else:
            cnt -= 1
    cnt += 1

text = text[cnt:]

In [132]:
# identify references
cnt = 0
while True:
    if "References" in text[cnt] or "Appendix" in text[cnt] or "Bibliography" in text[cnt]:
        if len(text[cnt].split()) < 3:
            break
    cnt += 1

text = text[:cnt]

In [133]:
# remove equation number references
clean1 = []
for t in text:
    if len(t.split()) == 1:
        if re.match("^\(\d\)", t):
            pass
        if re.match("\d", t):
            pass
    elif len(t.split()) == 0:
        pass
    else:
        clean1.append(t)

In [149]:
# remove headers
clean2 = []
cnt = 0
while cnt < len(clean1):
    if len(clean1[cnt].split()) < 10:
        dummy = re.sub("\d", "", clean1[cnt])
        dummy = dummy.strip()
        if dummy[-1] not in string.punctuation:
            dummy2 = re.sub("\d", "", clean1[cnt+1])
            dummy2 = dummy2.strip()
            if dummy2[0].isupper():
                pass
            else:
                clean2.append(clean1[cnt])
        else:
            clean2.append(clean1[cnt])
    else:
        clean2.append(clean1[cnt])
    cnt += 1

In [151]:
# remove citations
clean3 = " ".join(c for c in clean2)
clean3 = re.sub("^\[\d\]", "", clean3)
clean3 = re.sub("^\[\d\d\]", "", clean3)
clean3 = re.sub("^\[\d\d\]", "", clean3)
clean3 = re.sub("\n", " ", clean3)

In [152]:
clean3

'I describe an optimal control view of adversarial machine learning, where the dynamical system is the machine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s goals to do harm and be hard to detect. This view encompasses many types of adversarial machine learning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view encourages adversarial machine learning researcher to utilize advances in control theory and reinforcement learning. Machine learning has its mathematical foundation in concentration inequalities. This is a consequence of the independent and identically-distributed (i.i.d.) data assumption. In contrast, I suggest that adversarial machine learning may adopt optimal control as its mathematical foundation [3, 25]. There are telltale signs: adversarial attacks tend to be subtle and have peculiar non-i.i.d. structures – as control input might be. I will focus on deterministic discrete-time 

In [86]:
re.match("^\[\d\d\]", "[23]")

<re.Match object; span=(0, 4), match='[23]'>

In [93]:
thing = ' '
len(thing.split())

0

In [98]:
clean1

[' \n \n]',
 '.',
 'An Optimal Control View of Adversarial Machine Learning',
 'Department of Computer Sciences, University of Wisconsin-Madison',
 'Xiaojin Zhu',
 'Abstract',
 'I describe an optimal control view of adversarial machine learning, where the dynamical system is the\nmachine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s\ngoals to do harm and be hard to detect. This view encompasses many types of adversarial machine\nlearning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view\nencourages adversarial machine learning researcher to utilize advances in control theory and reinforcement\nlearning.',
 '1 Adversarial Machine Learning is not Machine Learning',
 'Machine learning has its mathematical foundation in concentration inequalities. This is a consequence of\nthe independent and identically-distributed (i.i.d.) data assumption. In contrast, I suggest that adversarial\nmachine learn

In [63]:
text

[' ',
 ' ',
 ' \n \n]',
 '.',
 'An Optimal Control View of Adversarial Machine Learning',
 'Department of Computer Sciences, University of Wisconsin-Madison',
 'Xiaojin Zhu',
 'Abstract',
 'I describe an optimal control view of adversarial machine learning, where the dynamical system is the\nmachine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s\ngoals to do harm and be hard to detect. This view encompasses many types of adversarial machine\nlearning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view\nencourages adversarial machine learning researcher to utilize advances in control theory and reinforcement\nlearning.',
 '1 Adversarial Machine Learning is not Machine Learning',
 'Machine learning has its mathematical foundation in concentration inequalities. This is a consequence of\nthe independent and identically-distributed (i.i.d.) data assumption. In contrast, I suggest that adversarial\nm

In [38]:
content[0].split("\n\n")

['8\n1\n0\n2',
 ' ',
 'v\no\nN\n1\n1',
 ' ',
 ' \n \n]',
 'G\nL\n.\ns\nc\n[\n \n ',
 '1\nv\n2\n2\n4\n4\n0',
 '.',
 '1\n1\n8\n1\n:\nv\ni\nX\nr\na',
 'An Optimal Control View of Adversarial Machine Learning',
 'Department of Computer Sciences, University of Wisconsin-Madison',
 'Xiaojin Zhu',
 'Abstract',
 'I describe an optimal control view of adversarial machine learning, where the dynamical system is the\nmachine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s\ngoals to do harm and be hard to detect. This view encompasses many types of adversarial machine\nlearning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view\nencourages adversarial machine learning researcher to utilize advances in control theory and reinforcement\nlearning.',
 '1 Adversarial Machine Learning is not Machine Learning',
 'Machine learning has its mathematical foundation in concentration inequalities. This is a consequence

In [18]:
text

[' ',
 ' ',
 ' \n \n]',
 '.',
 'An Optimal Control View of Adversarial Machine Learning',
 'Department of Computer Sciences, University of Wisconsin-Madison',
 'Xiaojin Zhu',
 'Abstract',
 'I describe an optimal control view of adversarial machine learning, where the dynamical system is the\nmachine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s\ngoals to do harm and be hard to detect. This view encompasses many types of adversarial machine\nlearning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view\nencourages adversarial machine learning researcher to utilize advances in control theory and reinforcement\nlearning.',
 '1 Adversarial Machine Learning is not Machine Learning',
 'Machine learning has its mathematical foundation in concentration inequalities. This is a consequence of\nthe independent and identically-distributed (i.i.d.) data assumption. In contrast, I suggest that adversarial\nm

In [ ]:
# if "abstract" header does not exist, use this block to extract abstract and keywords
if not abstract_string:
    stop = 0
    # find break point to stop at introduction
    while True:
        if "introduction" in text[stop].lower():
            break
        else:
            stop += 1
    cnt = 0
    # iterate through each line up to introduction
    while cnt < stop:
        # if keywords, put this in keywords variable
        if "keywords" in text[cnt].lower() or "key-words" in text[cnt].lower() or "keyword" in text[cnt].lower() or "key word" in text[cnt].lower():
            if len(text[cnt].split()) < 2:
                cnt += 1
            keywords_string += text[cnt]
        # if a number or a short piece of redundant text
        elif len(text[cnt].split()) < 2:
            pass
        # if ocr has picked up annoying numebrs along side with many "\n"
        elif text[cnt].count("\n") >= 3:
            dummy = text[cnt].split("\n")
            dummy_cnt = 0
            for d in dummy:
                if len(d)>1:
                    dummy_cnt += 1
            if dummy_cnt > 2:
                abstract_string += text[cnt]+" " 
        else:
            abstract_string += text[cnt]+" "
        cnt += 1
# block to clean up keywords if it has appended extra text to the start
keywords_string_2 = ""
if "\n" in keywords_string:
    keywords_dummy = keywords_string.split("\n")
    if "keywords" in keywords_dummy[0].lower() or "key-words" in keywords_dummy[0].lower() or "keyword" in keywords_dummy[0].lower() or "key word" in keywords_dummy[0].lower():
        pass
    else:
        cnt = 0
        while cnt < len(keywords_dummy):
            if "keywords" in keywords_dummy[cnt].lower() or "key-words" in keywords_dummy[cnt].lower() or "keyword" in keywords_dummy[cnt].lower() or "key word" in keywords_dummy[cnt].lower():
                if len(keywords_dummy[cnt].split()) < 2:
                    cnt += 1
                keywords_string_2 += keywords_dummy[cnt]
            cnt += 1
        keywords_string = keywords_string_2

In [12]:
content[0].split("\n\n")

['8\n1\n0\n2',
 ' ',
 'v\no\nN\n1\n1',
 ' ',
 ' \n \n]',
 'G\nL\n.\ns\nc\n[\n \n ',
 '1\nv\n2\n2\n4\n4\n0',
 '.',
 '1\n1\n8\n1\n:\nv\ni\nX\nr\na',
 'An Optimal Control View of Adversarial Machine Learning',
 'Department of Computer Sciences, University of Wisconsin-Madison',
 'Xiaojin Zhu',
 'Abstract',
 'I describe an optimal control view of adversarial machine learning, where the dynamical system is the\nmachine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s\ngoals to do harm and be hard to detect. This view encompasses many types of adversarial machine\nlearning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view\nencourages adversarial machine learning researcher to utilize advances in control theory and reinforcement\nlearning.',
 '1 Adversarial Machine Learning is not Machine Learning',
 'Machine learning has its mathematical foundation in concentration inequalities. This is a consequence